In [0]:
import re
#Define o nivel de log do spark
spark.sparkContext.setLogLevel("ERROR")
#Define o diretorio de origem dos dados
volume = "/Volumes/data_source/data_csv/tables_csv/"
files = dbutils.fs.ls(volume)
#função para limpar caracteres especiais do nome das colunas
def limpa_caracteres(col):
    return re.sub(r'[:;,\{\}\(\)\n\t=\s]', '_', col)
#Itera sob os arquivos e os salva na camada raw
for file in files:
    #recupera apenas o nome do arquivo
    nm_arquivo = file.name
    #le o arquivo em csv
    df = spark.read.format("csv").load(f"/Volumes/data_source/data_csv/tables_csv/{nm_arquivo}",header=True)
    #Armazena o nome das colunas ema lista
    nm_nova_colunas = [limpa_caracteres(col) for col in df.columns]
    #renomeia as colunas no dataframe
    df_colunas = df.toDF(*nm_nova_colunas)
    #Remove a extenção .csv para salvar no diretorio
    nm_arquivo = nm_arquivo.replace(".csv","")
    #Salva o arquivo na camada raw
    df_colunas.coalesce(1).write.format("parquet").mode("overwrite").save(f"/Volumes/raw/vendas/{nm_arquivo}")


In [0]:
#Itera sob os arquivos e os salva na camada raw
for file in files:
    nm_arquivo = file.name.replace(".csv","")
    #Remove arquivos de log do spark dos destinos
    caminho_destino = f"/Volumes/raw/vendas/{nm_arquivo}/"
    arquivos_temp = dbutils.fs.ls(caminho_destino)
    # Filtrar e excluir arquivos que começam com _started_ ou _committed_
    for arquivo in arquivos_temp:
        if arquivo.name.startswith("_started_") or arquivo.name.startswith("_committed_") or arquivo.name.startswith("_SUCCESS"):
            dbutils.fs.rm(arquivo.path)